CellChat inference and analysis of spatially proximal cell-cell communication from spatially resolved transcriptomics (multiple datasets)

In [17]:
ptm = Sys.time()

library(CellChat)
library(Seurat)
library(patchwork)
library(ComplexHeatmap)
library(grid)
options(stringsAsFactors = FALSE)

Warning message:
“package ‘ComplexHeatmap’ was built under R version 4.2.1”
Loading required package: grid

ComplexHeatmap version 2.14.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))




In [53]:
results_folder ='/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/cellTrek/cellchat/'

In [54]:
#cellTrekObs <- readRDS(paste0(ov_visium, "ov_cellTrek_obj.rds"))
#cell2loc = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'
#adata_vis_match <- zellkonverter::readH5AD(file.path(cell2loc, "final_adata_vis_match.h5ad"))

cell2loc = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'
#ovTrainAll <- readRDS(paste0(cell2loc, "cellTrek/celltrek_results_ntree1000_p500_celltrek.rds"))
ovCellTrekList <- readRDS(paste0(cell2loc, "cellTrek/celltrek_results_updated.rds"))

For one sample at a time

In [ ]:
ovCellTrek <- adata_vis_match[[1]]
# 1. Extract expression and meta
data.input <- GetAssayData(ovCellTrek, slot = "data")  # or "counts"
meta <- ovCellTrek@meta.data
# 2. Create CellChat object
cellchat <- CellChat::createCellChat(object = data.input, meta = meta, group.by = "cell_type")
CellChatDB <- CellChatDB.mouse  # or .human
cellchat@DB <- CellChatDB
cellchat <- subsetData(cellchat)
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- computeCommunProb(cellchat)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)
# all predicted interactions (data.frame)
comm <- subsetCommunication(cellchat)
head(comm)
# save
write.csv(comm, file = file.path(results_folder, "ov1_cellchat_all_interactions.csv"), row.names = FALSE)
# list pathways and group sizes
signaling_paths <- names(cellchat@netP)
print(signaling_paths)
groupSize <- as.numeric(table(cellchat@idents))
groupSize


For all in a loop

In [55]:

cellchat_list <- list()

for (i in seq_along(ovCellTrekList)) {
  
  sample_name <- if (!is.null(names(ovCellTrekList)) && names(ovCellTrekList)[i] != "") {
    names(ovCellTrekList)[i]
  } else {
    paste0("sample_", i)
  }
  message("Creating CellChat object for ", sample_name)
  
  ovCellTrek <- ovCellTrekList[[i]]
  
  # 1. Extract expression & metadata
  data.input <- GetAssayData(ovCellTrek, slot = "data")
  meta <- ovCellTrek@meta.data
  
  # Ensure you have a grouping column (rename if needed)
  # e.g. your column might be "cell_type_collapsed"
  if (!"cell_type" %in% colnames(meta) && "cell_type_collapsed" %in% colnames(meta)) {
    meta$cell_type <- meta$cell_type_collapsed
  }
  
  # 2. Create CellChat object
  cellchat <- CellChat::createCellChat(
    object = data.input,
    meta = meta,
    group.by = "cell_type"
  )
  
  cellchat_list[[sample_name]] <- cellchat
}

# --- Optional: preview what was created ---
cellchat_list


Creating CellChat object for OV_1



[1] "Create a CellChat object from a data matrix"
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  B naive activated B naive resting CD4 CD8 Cycling Plasma Cells Dendritic Cells Endothelial Macrophage Mesenchymal Monocytes Neutrophils NK_cell Plasma Cells Th1 Th17 Treg Tumour 


Creating CellChat object for OV_2



[1] "Create a CellChat object from a data matrix"
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  B naive activated B naive resting CD4 CD8 Cycling Plasma Cells Dendritic Cells Endothelial Macrophage Mesenchymal Monocytes Neutrophils NK_cell Plasma Cells Th1 Th17 Treg Tumour 


Creating CellChat object for OV_3



...

Creating CellChat object for OV_5



[1] "Create a CellChat object from a data matrix"
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  B naive activated B naive resting CD4 CD8 Dendritic Cells Endothelial Macrophage Mesenchymal Monocytes Neutrophils NK_cell Plasma Cells Th1 Th17 Treg Tumour 


Creating CellChat object for OV_6



[1] "Create a CellChat object from a data matrix"
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  B naive activated B naive resting CD4 CD8 Cycling Plasma Cells Dendritic Cells Endothelial Macrophage Mesenchymal Monocytes Neutrophils NK_cell Plasma Cells Th1 Th17 Treg Tumour 


$OV_1
An object of class CellChat created from a single dataset 
 14542 genes.
 9276 cells. 
CellChat analysis of single cell RNA-seq data! 

$OV_2
An object of class CellChat created from a single dataset 
 14542 genes.
 13696 cells. 
CellChat analysis of single cell RNA-seq data! 

$OV_3
An object of class CellChat created from a single dataset 
 14542 genes.
 10291 cells. 
CellChat analysis of single cell RNA-seq data! 

$OV_4
An object of class CellChat created from a single dataset 
 14542 genes.
 5964 cells. 
CellChat analysis of single cell RNA-seq data! 

$OV_5
An object of class CellChat created from a single dataset 
 14542 genes.
 9080 cells. 
CellChat analysis of single cell RNA-seq data! 

$OV_6
An object of class CellChat created from a single dataset 
 14542 genes.
 5391 cells. 
CellChat analysis of single cell RNA-seq data! 


In [56]:
# Load mouse ligand-receptor database
CellChatDB <- CellChatDB.mouse
for (i in names(cellchat_list)) {
  cellchat_list[[i]]@DB <- CellChatDB
  cellchat_list[[i]] <- subsetData(cellchat_list[[i]])           # subset relevant genes
  cellchat_list[[i]] <- identifyOverExpressedGenes(cellchat_list[[i]])
  cellchat_list[[i]] <- identifyOverExpressedInteractions(cellchat_list[[i]])
  cellchat_list[[i]] <- computeCommunProb(cellchat_list[[i]])
  cellchat_list[[i]] <- computeCommunProbPathway(cellchat_list[[i]])
  cellchat_list[[i]] <- aggregateNet(cellchat_list[[i]])
}


Issue identified!! Please check the official Gene Symbol of the following genes:  
 H2-BI H2-Ea-ps 
triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2025-10-31 12:59:52]"
[1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2025-10-31 13:03:38]"
Issue identified!! Please check the official Gene Symbol of the following genes:  
 H2-BI H2-Ea-ps 
triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2025-10-31 13:04:21]"
[1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2025-10-31 13:08:28]"
Issue identified!! Please check the official Gene Symbol of the following genes:  
 H2-BI H2-Ea-ps 
triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2025-10-31 13:09:01]"
[1] ">>> CellCh

In [57]:

comm_list=list()

for (sample_name in names(cellchat_list)) {
  cellchat <- cellchat_list[[sample_name]]
  
  # Extract all significant interactions
  comm <- subsetCommunication(cellchat)
  
  # Save to CSV
  write.csv(
    comm,
    file = file.path(results_folder, paste0(sample_name, "_cellchat_all_interactions.csv")),
    row.names = FALSE
  )
  
  # Store in a list for later combination
  comm_list[[sample_name]] <- comm
}

# --- Optional: combine all into one dataframe ---
comm_all <- bind_rows(comm_list, .id = "sample")

# Quick preview
head(comm_all)

,sample,source,target,ligand,receptor,prob,pval,interaction_name,interaction_name_2,pathway_name,annotation,evidence
,<chr>,<fct>,<fct>,<chr>,<chr>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<chr>,<chr>
1,OV_1,B naive activated,B naive activated,Tgfb1,TGFbR1_R2,0.02177701,0,TGFB1_TGFBR1_TGFBR2,Tgfb1 - (Tgfbr1+Tgfbr2),TGFb,Secreted Signaling,KEGG: mmu04350
2,OV_1,B naive resting,B naive activated,Tgfb1,TGFbR1_R2,0.02292956,0,TGFB1_TGFBR1_TGFBR2,Tgfb1 - (Tgfbr1+Tgfbr2),TGFb,Secreted Signaling,KEGG: mmu04350
3,OV_1,CD4,B naive activated,Tgfb1,TGFbR1_R2,0.01804501,0,TGFB1_TGFBR1_TGFBR2,Tgfb1 - (Tgfbr1+Tgfbr2),TGFb,Secreted Signaling,KEGG: mmu04350
4,OV_1,CD8,B naive activated,Tgfb1,TGFbR1_R2,0.02235130,0,TGFB1_TGFBR1_TGFBR2,Tgfb1 - (Tgfbr1+Tgfbr2),TGFb,Secreted Signaling,KEGG: mmu04350
5,OV_1,Dendritic Cells,B naive activated,Tgfb1,TGFbR1_R2,0.01184832,0,TGFB1_TGFBR1_TGFBR2,Tgfb1 - (Tgfbr1+Tgfbr2),TGFb,Secreted Signaling,KEGG: mmu04350
6,OV_1,Endothelial,B naive activated,Tgfb1,TGFbR1_R2,0.01188053,0,TGFB1_TGFBR1_TGFBR2,Tgfb1 - (Tgfbr1+Tgfbr2),TGFb,Secreted Signaling,KEGG: mmu04350


Plot heatmap of interactions per sample 

In [43]:
plot_list <- lapply(names(cellchat_list), function(x){
    plot <- netVisual_heatmap(cellchat_list[[x]], measure="count", color.heatmap = "Blues")
      # Save each plot
 pdf(
    file = paste0(results_folder, x, "_heatmap.pdf"),
    width = 8,
    height = 6)
  ComplexHeatmap::draw(plot)
  dev.off()
  plot
})

names(plot_list) <- names(cellchat_list)

Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Do heatmap based on a single object 


Do heatmap based on a single object 


Do heatmap based on a single object 


Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”


In [58]:

cellchat_pre <- mergeCellChat(
  cellchat_list[1:3],
  add.names = names(cellchat_list[1:3]),
  cell.prefix = TRUE)

  cellchat_post <- mergeCellChat(
  cellchat_list[4:6],
  add.names = names(cellchat_list[4:6]),
  cell.prefix = TRUE)

Warning message in mergeCellChat(cellchat_list[1:3], add.names = names(cellchat_list[1:3]), :
“Prefix cell names!”


The cell barcodes in merged 'meta' is  ov16_AAAGTGAGTGGTACAG.1 ov17_AAGTGAAGTATGCAAA.1 ov20_AGAGAATCAAACTCTG.1 ov20_GAGTCTATCGCCGAGT.1 ov20_GATAGAAGTACACTCA.1 ov20_GGCTGTGTCTGCATGA.1 


Warning message in mergeCellChat(cellchat_list[1:3], add.names = names(cellchat_list[1:3]), :
“The cell barcodes in merged 'meta' is different from those in the used data matrix.
              We now simply assign the colnames in the data matrix to the rownames of merged 'mata'!”
Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.

Warning message in mergeCellChat(cellchat_list[4:6], add.names = names(cellchat_list[4:6]), :
“Prefix cell names!”


The cell barcodes in merged 'meta' is  ov18_CAGTGCGGTGACTCGC.1 ov18_GGGTTTATCGCATTAG.1 ov16_AGTGTTGGTGATACAA.1 ov17_AACAACCCAAAGCGTG.1 ov17_AAGCCATAGGTCACTT.1 ov17_CGCCAGATCCCGAATA.1 


Warning message in mergeCellChat(cellchat_list[4:6], add.names = names(cellchat_list[4:6]), :
“The cell barcodes in merged 'meta' is different from those in the used data matrix.
              We now simply assign the colnames in the data matrix to the rownames of merged 'mata'!”
Merge the following slots: 'data.signaling','images','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



In [59]:

#Make the heatmap object (counts of significant LR pairs between sender→receiver)
ht <- netVisual_heatmap(
  object        = cellchat_post,
  measure       = "count",
  color.heatmap = c("white", "steelblue")
)

# 3. Draw to screen
ComplexHeatmap::draw(ht)

# 4. Save to PDF
pdf(
  file = file.path(results_folder, "CellChat_Post_heatmap.pdf"),
  width = 9,
  height = 8
)
ComplexHeatmap::draw(ht)
dev.off()

Do heatmap based on a merged object 




ERROR: Error in obj2 - obj1: non-conformable arrays


In [60]:

#Make the heatmap object (counts of significant LR pairs between sender→receiver)
ht <- netVisual_heatmap(
  object        = cellchat_pre,
  measure       = "count",
  color.heatmap = c("white", "steelblue")
)

# 3. Draw to screen
ComplexHeatmap::draw(ht)

# 4. Save to PDF
pdf(
  file = file.path(results_folder, "CellChat_Pre_heatmap.pdf"),
  width = 9,
  height = 8
)
ComplexHeatmap::draw(ht)
dev.off()


Do heatmap based on a merged object 




Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”
Warning message:
“`legend_height` you specified is too small, use the default minimal
height.”


png 
  2

previous attempts below

In [ ]:
adata_vis_match[[1]]@assays$RNA@data[1:5,1:5]

In [ ]:
distMats <- lapply(cellTrekObs, function(ctOb){
    obCoord <- as.matrix(ctOb@meta.data[,c("coord_x", "coord_y")])
    obDist <- CellChat::computeRegionDistance(obCoord,
                                              meta = ctOb@meta.data,
                                              contact.knn.k=10)
    return(obDist)
})

The above is seeing if it will work if I load Ollie's version unprocessed, still no result

In [ ]:
#local
cell2loc = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/cellTrek/'
ov_visium = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/cellTrek/'

#### Load Visium Data
ovVis <- readRDS(paste0(ov_visium, "celltrek_results.rds"))  #version of Ollie's filtered against mine

#### Load scRNA-seq
ovSc <- readRDS("/home/lythgo02/Documents/OV_visium/upk10_sc_400_newlyAnnot.rds")

In [ ]:
#convert updated annotation to factor with levels organised in alphabetical order for plotting purposes 
ovVis <- lapply(ovVis, function(i){
    i$cell_type <- ifelse(grepl("Tumour", i$updated_annotation), "Tumour",  #collapse tumour 1,2,3 subgroups into one 
                                    i$updated_annotation)  
    i$cell_type <- factor(i$cell_type,
                                    levels=sort(unique(i$cell_type)))                                                        
    return(i)
})

In [ ]:

ovVis <- lapply(ovVis, function(ovCellTrek) {
    cell_type <- as.character(ovCellTrek@meta.data$cell_type)

    #merge categories with consistently low counts across samples
    cell_type[grepl("Th", cell_type)] <- "T-helper"
    cell_type[grepl("Cycling", cell_type)] <- "Plasma Cells"
    # Add to metadata
    ovCellTrek@meta.data$cell_type_collapsed <- factor(cell_type)

    return(ovCellTrek)
})

In [ ]:
ov1 <- ovVis[[1]]
colnames(ov1@meta.data)
unique(ov1@meta.data$cell_type) #from celltrek
summary(ov1@meta.data$coord_x) #dimensions in pixels so will need converting 
ov1@images$slice1@scale.factors
ov1@images$slice1@spot.radius
slotNames(ov1[["RNA"]])


Cellchat requires normalised data
 - Ollie appears to have previously used SCTransform which is from the same authors as Cellchat (Seurat lot)
 - In their workflow they appear to use SCT data but this is the residuals after regression (like Z-scaled), given they say to use log normalised data, not scaled, will use NormaliseData function 
 - I am using the output from celltrek as input for cellchat so the normalised data is in obj[["RNA]]@data

Prepare inputs
CHECK CONVERSION OF PIXEL TO UM - CHCK THE LOGIC
 - extract normalised expression values 
 - create metadataframe with labels and treatments 
 - convert spot pixel coordinates to um for spatial.locs

In [ ]:
ovVis_input <- lapply(ovVis, function(x) {
  # Extract normalized expression matrix
  data.input <- GetAssayData(x, assay = "RNA", slot = "data")
  
  # Use full metadata from Seurat object directly for CellChat
  meta.input <- x@meta.data
  
  # Convert pixel coordinates to microns
  spot.size <- 65  # theoretical spot size in µm
  pixel.diam <- x@images$slice1@scale.factors$spot_dis  # spot diameter in pixels
  conversion.factor <- spot.size / pixel.diam           # µm per pixel
  
  spatial.locs <- as.matrix(meta.input[, c("coord_x", "coord_y")]) * conversion.factor
  colnames(spatial.locs) <- c("x", "y")
  
  scale.factors <- list(
  spot.diameter = 65,
  spot = x@images$slice1@scale.factors$spot)  # 'spot' is the pixel diameter at full-res
  
  # QC: compute nearest-neighbor distances
  d.spatial <- computeCellDistance(coordinates = spatial.locs,
                                   ratio = 1, tol = spot.size/2)
  cat("Minimum NN distance (µm):", min(d.spatial[d.spatial != 0]), "\n")
  
  # Return list with expression, metadata, spatial coordinates
  list(data = data.input, meta = meta.input, spatial = spatial.locs, scale.factors=scale.factors)
})



Visium spots:
 - The center-to-center distance between spots is ~100 µm (this comes from the array design: ~65 µm spot diameter, ~100 µm spacing).
 - If you were plotting spot coordinates directly, the nearest-neighbor distances between spots should be ~100 µm.
 - CellTrek interpolates cells within each spot. Multiple cells can occupy a single spot or nearby positions so the NN distances between cells are smaller — ~10–20 µm; cell-to-cell rather than spot-to-spot 
 - ~100 µm = nearest spots in Visium
 - ~14 µm = nearest mapped cells after CellTrek

In [ ]:
head(ovVis_input$OV_1$meta)
head(ovVis[[1]]@meta.data[, c("coord_x", "coord_y")])  # raw pixel coords
head(ovVis_input$OV_1$spatial)  
head(ovVis_input$OV_1$data)
ovVis_input$OV_1$scale.factors

In [ ]:
ov1 <- ovVis_input[[1]]
colnames(ov1$meta)
all(rownames(meta) == rownames(coordinates))
table(meta$group)
nrow(ov1$spatial)

In [ ]:
CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)

Pick the subset of the database that you actually want to use

In [ ]:

# use a subset of CellChatDB for cell-cell communication analysis
CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling") # use Secreted Signaling
# use all CellChatDB for cell-cell communication analysis
# CellChatDB.use <- CellChatDB # simply use the default CellChatDB

In [ ]:


cellchat_list <- lapply(names(ovVis_input), function(sample_name) {
  sample_data <- ovVis_input[[sample_name]]
  
  cellchat <- CellChat::createCellChat(
    object = sample_data$data,
    meta = sample_data$meta,
    group.by = "cell_type_collapsed",                 # adjust to the column name in meta for cell types
    coordinates = sample_data$spatial,
    datatype = "spatial",
    #scale.factors = sample_data$scale.factors,
    spatial.factors = data.frame(ratio = 1, tol = 32.5),  # adjust tol if needed
    do.sparse=TRUE
  )
  
  # Set CellChat database (optional: replace with the specific database you want to use)
  cellchat@DB <- CellChatDB.mouse  # or CellChatDB.use if you loaded a custom DB
  
  return(cellchat)
})

names(cellchat_list) <- names(ovVis_input)



In [ ]:
cellchat <- cellchat_list[[1]]
levels(cellchat@idents)
groupSize <- as.numeric(table(cellchat@idents)) # cells per type
groupSize
slotNames(cellchat) #data.raw is only populated if you provide the raw counts which aren't actually required by cellchat



In [ ]:

library(presto)

# Subset data to relevant genes and ligand-receptor pairs
cellchat <- subsetData(cellchat)

In [ ]:
cellchat@spatial


In [ ]:

cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- computeCommunProb(cellchat, raw.use = FALSE)
cellchat <- filterCommunication(cellchat, min.cells = 10)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)
